# 1. Set up

In [5]:
# from google.colab import drive
# drive.mount('/content/gdrive')

import re
import json
import gensim.downloader as api

!pip install langid
import langid

!pip install gensim
from gensim.models import Word2Vec

!pip install -U spacy
!python -m spacy download it_core_news_sm
import spacy

!pip install sense2vec
from sense2vec import Sense2Vec

import random

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sense2vec 1.0.2 requires spacy<3.0.0,>=2.2.3, but you have spacy 3.7.4 which is incompatible.



  Using cached spacy-3.7.4-cp39-cp39-win_amd64.whl.metadata (27 kB)
  Using cached thinc-8.2.3-cp39-cp39-win_amd64.whl.metadata (15 kB)
  Using cached srsly-2.4.8-cp39-cp39-win_amd64.whl.metadata (20 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl.metadata (14 kB)
Using cached spacy-3.7.4-cp39-cp39-win_amd64.whl (12.2 MB)
Using cached catalogue-2.0.10-py3-none-any.whl (17 kB)
Using cached srsly-2.4.8-cp39-cp39-win_amd64.whl (483 kB)
Using cached thinc-8.2.3-cp39-cp39-win_amd64.whl (1.5 MB)
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.2
    Uninstalling catalogue-1.0.2:
      Successfully uninstalled catalogue-1.0.2
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.7
    Uninstalling srsly-1.0.7:
      Successfully uninstalled srsly-1.0.7
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.6
    Uninstalling thinc-7.4.6:
      Successfully uninstalled thinc-7.4.6
  Attempting uninstall: spacy
    Found

'wget' is not recognized as an internal or external command,
operable program or batch file.
tar: Error opening archive: Failed to open 's2v_reddit_2015_md.tar.gz'


  Using cached spacy-2.3.9-cp39-cp39-win_amd64.whl.metadata (16 kB)
  Using cached thinc-7.4.6-cp39-cp39-win_amd64.whl.metadata (23 kB)
  Using cached srsly-1.0.7-cp39-cp39-win_amd64.whl.metadata (14 kB)
  Using cached catalogue-1.0.2-py2.py3-none-any.whl.metadata (13 kB)
Using cached spacy-2.3.9-cp39-cp39-win_amd64.whl (9.1 MB)
Using cached catalogue-1.0.2-py2.py3-none-any.whl (16 kB)
Using cached srsly-1.0.7-cp39-cp39-win_amd64.whl (358 kB)
Using cached thinc-7.4.6-cp39-cp39-win_amd64.whl (830 kB)
  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.8
    Uninstalling srsly-2.4.8:
      Successfully uninstalled srsly-2.4.8
  Attempting uninstall: catalogue
    Found existing installation: catalogue 2.0.10
    Uninstalling catalogue-2.0.10:
      Successfully uninstalled catalogue-2.0.10
  Attempting uninstall: thinc
    Found existing installation: thinc 8.2.3
    Uninstalling thinc-8.2.3:
      Successfully uninstalled thinc-8.2.3
  Attempting uninstall: spacy
  

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
confection 0.1.4 requires srsly<3.0.0,>=2.4.0, but you have srsly 1.0.7 which is incompatible.
it-core-news-sm 3.7.0 requires spacy<3.8.0,>=3.7.0, but you have spacy 2.3.9 which is incompatible.
weasel 0.3.4 requires srsly<3.0.0,>=2.4.3, but you have srsly 1.0.7 which is incompatible.


ImportError: cannot import name 'component' from 'spacy' (c:\Users\35193\miniconda3\envs\cuda_env\lib\site-packages\spacy\__init__.py)

# Find Distractores


1. Sense2Vec (https://jovian.com/sunilbabu1981/qg-colab-1-01-generating-distractors-using-sense2vec)

In [15]:
from sense2vec import Sense2Vec

# Download a pre-trained Sense2Vec model
!tar -xvf s2v_reddit_2015_md.tar.gz

# Load the Sense2Vec model
s2v_model_path = "s2v_old"
s2v = Sense2Vec().from_disk(s2v_model_path)

x ./._s2v_old
x ./s2v_old/
x ./s2v_old/._freqs.json
x ./s2v_old/freqs.json
x ./s2v_old/._vectors
x ./s2v_old/vectors
x ./s2v_old/._cfg
x ./s2v_old/cfg
x ./s2v_old/._strings.json
x ./s2v_old/strings.json
x ./s2v_old/._key2row
x ./s2v_old/key2row


In [4]:
# load the sense2vec vectors using the comments from reddit
from sense2vec import Sense2Vec
s2v = Sense2Vec().from_disk('s2v_old')

In [12]:
# Steps to find distractors for "Donald Trump"
word = 'esercizio'
# Convert the string to lower case letters
word = word.lower()
# Replace space with underscore(_)
word = word.replace(' ','_')
print(word)

esercizio


In [13]:
# Find the distractors by feeding the word to sense2vec
sense = s2v.get_best_sense(word)
# Get the sense
print("Best Sense",sense)

Best Sense None


In [14]:
# Now find the most similar words for this word in this sense
most_similar = s2v.most_similar(sense,n=12)
print(most_similar)

TypeError: 'NoneType' object is not iterable

In [8]:
# Packaging this into a callable function
from collections import OrderedDict
def sense2Vec_getSimilarWords(word,s2v):
  # Declare the output and its type - arrays or list
  similarWords = []

  # String processing - converting to lower and introducing a underscore for blanks
  word = word.lower().replace(' ','_')

  # Get the best sense for the given word
  sense = s2v.get_best_sense(word)
  most_Similar = s2v.most_similar(sense,n=20)

  for each_word in most_Similar:
    similarWords.append(each_word[0].split("|")[0].replace("_"," "))

  return similarWords

In [11]:
sense2Vec_getSimilarWords("Natural Language Processing",s2v)

TypeError: 'NoneType' object is not iterable

- Problem needs training only works forenglish words

## Insert distractors into the dataset

In [41]:
def find_distractors(hypernym, data, term, num_distractors):
  # sense = s2v_model.get_best_sense(hypernym)
  # if sense:
  #   distractors = s2v_model.most_similar(sense, n=5)
  #   filtered_distractors = [distractor[0].split("|")[0].replace("_", " ") for distractor in distractors if distractor[0].split("|")[0].replace("_", " ") not in data[term]]
  #   return filtered_distractors[:topn]
  # else:
  distractors = w2v_model.wv.most_similar(hypernym, topn = 5)
  filtered_distractors = [distractor[0] for distractor in distractors if distractor[0] not in data[term]]
  return filtered_distractors[:num_distractors]

def save_jsonl(file_path, data):
  id_seq = 0
  with open(file_path, "w") as output_file:
    for term, hypernyms in data.items():
      for hypernym in hypernyms:
        distractors = find_distractors(hypernym, data, term, num_distractors = 3)
        entries = (hypernym, *distractors)
        choices = list(entries)
        random.shuffle(choices) # to create randomness
        reformatted_json_data = {
              'id' : id_seq,
              'text': term,
              'choices': choices,
              'label' : choices.index(hypernym)
        }
        json.dump(reformatted_json_data, output_file)
        output_file.write("\n")
        id_seq +=1

def read_lines_jsonl(file_path, num_lines):
  with open(file_path, 'r') as f:
    json_list = list(f)
    for line in json_list[:num_lines]:
      data = json.loads(line)
      print(data)

In [42]:
# train jsonl file
save_jsonl("hypernym_discovery-task26-train-data.jsonl", train_hypernyms)
read_lines_jsonl("hypernym_discovery-task26-train-data.jsonl", num_lines = 10) # preview of the first 10 lines

{'id': 0, 'text': 'sesto', 'choices': ['negativizzazione', 'alternare', 'grado', 'formicolio'], 'label': 2}
{'id': 1, 'text': 'sesto', 'choices': ['immagazzinare', 'autenticità', 'solubile', 'numero ordinale'], 'label': 3}
{'id': 2, 'text': 'sesto', 'choices': ['elettromagnetico', 'frazione', 'cristiano', 'serenza'], 'label': 1}
{'id': 3, 'text': 'sesto', 'choices': ['carica', 'glicerina', 'estorsione', 'segnalare'], 'label': 0}
{'id': 4, 'text': 'Sigillo', 'choices': ['collaborazionismo', 'comune', 'contrattazione', 'sembianza'], 'label': 1}
{'id': 5, 'text': 'Sigillo', 'choices': ['perfezionismo', 'municipalità', 'console per videogiochi', 'viticoltore'], 'label': 1}
{'id': 6, 'text': 'Sigillo', 'choices': ['tagliente', 'comune italiano', 'aggressivamente', 'bellah'], 'label': 1}
{'id': 7, 'text': 'Sigillo', 'choices': ['elettromagnetico', 'serenza', 'cristiano', 'frazione'], 'label': 3}
{'id': 8, 'text': 'Sigillo', 'choices': ['subcosciente', 'pedagogista', 'imbattibile', 'paese'], 

In [43]:
# test jsonl file
save_jsonl("hypernym_discovery-task26-test-data.jsonl", test_hypernyms)
read_lines_jsonl("hypernym_discovery-task26-train-data.jsonl", num_lines = 10) # preview of the first 10 lines

{'id': 0, 'text': 'sesto', 'choices': ['negativizzazione', 'alternare', 'grado', 'formicolio'], 'label': 2}
{'id': 1, 'text': 'sesto', 'choices': ['immagazzinare', 'autenticità', 'solubile', 'numero ordinale'], 'label': 3}
{'id': 2, 'text': 'sesto', 'choices': ['elettromagnetico', 'frazione', 'cristiano', 'serenza'], 'label': 1}
{'id': 3, 'text': 'sesto', 'choices': ['carica', 'glicerina', 'estorsione', 'segnalare'], 'label': 0}
{'id': 4, 'text': 'Sigillo', 'choices': ['collaborazionismo', 'comune', 'contrattazione', 'sembianza'], 'label': 1}
{'id': 5, 'text': 'Sigillo', 'choices': ['perfezionismo', 'municipalità', 'console per videogiochi', 'viticoltore'], 'label': 1}
{'id': 6, 'text': 'Sigillo', 'choices': ['tagliente', 'comune italiano', 'aggressivamente', 'bellah'], 'label': 1}
{'id': 7, 'text': 'Sigillo', 'choices': ['elettromagnetico', 'serenza', 'cristiano', 'frazione'], 'label': 3}
{'id': 8, 'text': 'Sigillo', 'choices': ['subcosciente', 'pedagogista', 'imbattibile', 'paese'], 

# 3. Prompt formulation

In [44]:
prompts = [
    "Il termine '{{text}}' può essere iperonimo di: {{choices}}",
    "Dato il termine '{{text}}', quale tra le seguenti parole è un suo iperonimo? {{choices}}",
    "Scegli l'iperonimo del termine ''{{text}}: {{choices}}"
]

In [45]:
def save_prompts_jsonl(prompts, file_path):
  json_prompts = []
  for prompt in prompts:
    json_prompts.append({"prompt": prompt})

  with open(file_path, "w") as output_file:
    for json_prompt in json_prompts:
      json.dump(json_prompt, output_file)
      output_file.write("\n")

In [46]:
save_prompts_jsonl(prompts, "hypernym_discovery-task26-json.jsonl")
read_lines_jsonl("hypernym_discovery-task26-json.jsonl", num_lines = 3)

{'prompt': "Il termine '{{text}}' può essere iperonimo di: {{choices}}"}
{'prompt': "Dato il termine '{{text}}', quale tra le seguenti parole è un suo iperonimo? {{choices}}"}
{'prompt': "Scegli l'iperonimo del termine ''{{text}}: {{choices}}"}


# 4. Llama 2

In [ ]:
# Device agnostic code
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
# Configuration
runtimeFlag = device
cache_dir = None
scaling_factor = 1.0

In [ ]:
!pip install -q -U transformers peft accelerate optimum

!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.9/409.9 kB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
!pip install -q -U pdfminer.six # could maybe add pre-built wheels to speed this up.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 56.7 MB/s eta 0:00:00


In [ ]:
# https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
# Load Model
import transformers
import torch
import json
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_id = "MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7"
tokenizer_llama = AutoTokenizer.from_pretrained(model_id)
model_llama = AutoModelForSequenceClassification.from_pretrained(model_id).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

## Perfomance Evaluation with Llama 2

In [ ]:
import json
import torch
import torch.nn.functional as F

with open('train_dataset.json', 'r') as file:
    entities = json.load(file)[:6]
    for entity in entities:
      print(entity)
      id = entity['id']
      text = entity['text']
      choices = entity['choices']
      label = entity['label']

      prompt_formatted = f"Il termine '{text}' può essere iperonimo di: {choices}"
      input = tokenizer_llama(prompt_formatted, return_tensors="pt").to(device)
      output = model_llama(**input)

      probabilities = F.softmax(output.logits, dim=-1)
      probabilities = probabilities.squeeze().tolist()

      # # Find the index of the choice with the highest probability
      # for choice, prob in zip(choices, probabilities):
      #       print(f"Choice: {choice}, Probability: {prob}\n")

      logits = output.logits[0].detach().cpu().numpy()

      probabilities = torch.softmax(torch.tensor(logits), -1).detach().cpu().numpy()
      print(f"Probabilities: {probabilities}")
      prediction_index = probabilities.argmax()
      prediction_label = choices[prediction_index]
      prediction_confidence = probabilities[prediction_index] * 100
      prediction = {"label": prediction_index, "confidence": round(prediction_confidence, 1)}

      is_correct = prediction_index == label

      print("Prompt:", prompt_formatted)
      print("Prediction:", prediction)
      print("Actual Label:", label)
      print("Correct Prediction:", is_correct)
      print()

{'id': 0, 'text': 'sesto', 'choices': ['grado', 'casa reale', 'fulminazione', 'confetto'], 'label': 0}
Probabilities: [5.749090e-04 9.990707e-01 3.544809e-04]
Prompt: Il termine 'sesto' può essere iperonimo di: ['grado', 'casa reale', 'fulminazione', 'confetto']
Prediction: {'label': 1, 'confidence': 99.9}
Actual Label: 0
Correct Prediction: False

{'id': 0, 'text': 'sesto', 'choices': ['numero ordinale', 'innovazione', 'specialmente', 'casalmaggiore'], 'label': 0}
Probabilities: [4.6423267e-04 9.9934262e-01 1.9315351e-04]
Prompt: Il termine 'sesto' può essere iperonimo di: ['numero ordinale', 'innovazione', 'specialmente', 'casalmaggiore']
Prediction: {'label': 1, 'confidence': 99.9}
Actual Label: 0
Correct Prediction: False

{'id': 0, 'text': 'sesto', 'choices': ['frascheggiare', 'frazione', 'particella', 'avvinghiare'], 'label': 1}
Probabilities: [8.4733742e-04 9.9877518e-01 3.7740712e-04]
Prompt: Il termine 'sesto' può essere iperonimo di: ['frascheggiare', 'frazione', 'particella'

# 5. Roberta

In [ ]:
import json
import torch
from torch.distributions import Categorical
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("LIAMF-USP/roberta-large-finetuned-race")
model = AutoModelForSequenceClassification.from_pretrained("LIAMF-USP/roberta-large-finetuned-race")

tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at LIAMF-USP/roberta-large-finetuned-race and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
